In [ ]:
## Setup 
# import modules
import pandas as pd
import os
import pyodbc
from datetime import datetime
import arcpy
import numpy as np
import urllib
#import geojson
#import json
#from urllib.request import urlopen
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
#Test 2
# setup workspace folder
workspace = "//Trpa-fs01/gis/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/"

# setup environment variables
arcpy.env.overwriteOutput = True
arcpy.env.workspace = "//Trpa-fs01/GIS/PROJECTS/ResearchAnalysis/Monitoring/Data/Crash/CrashData/CrashData.gdb"

# create a spatial reference object for the output coordinate system 
# output projection for data going into SDE should be UTM Zone 10N (EPSG: 26910)
out_coordinate_system = arcpy.SpatialReference(26910)

# network path to connection files
filePath = "//Trpa-fs01/GIS/DB_CONNECT"

# database file path 
sdeBase  = os.path.join(filePath, "Vector.sde")

# SDE feature classes needed for spatial joins
corridor = os.path.join(sdeBase, 'sde.SDE.Transportation\sde.SDE.Corridor')
trpa     = os.path.join(sdeBase, 'sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy')

# define csv lat/long field names for xy table to point
x_coords = 'POINT_X'
y_coords = 'POINT_Y'

# SDE feature class to update
crashSDE  = os.path.join(sdeBase, 'sde.SDE.Transportation\sde.SDE.Highway_Collisions')

# Get Crash Data
caCrashes = os.path.join(workspace, "BothCACounties_Crashes_1320_Unclean_v2.csv")
dfCACrash = pd.read_csv(caCrashes)

nvCrashes = os.path.join(workspace, "NV_LAKE TAHOE BASIN - ALL ROADS 2013-2020.csv")
dfNVCrash = pd.read_csv(nvCrashes)
    
# # in memory files
memory = "memory" + "\\"

# replaces features in outfc with exact same schema
def updateSDE(inputfc,outfc, fieldnames):
        # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sde, "ALL")

    # deletes all rows from the SDE feature class
    arcpy.TruncateTable_management(outfc)
    print ("\nDeleted all records in: {}\n".format(outfc))

    # insert rows from Temporary feature class to SDE feature class
    with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
        count = 0
        with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
            for row in iCursor:
                oCursor.insertRow(row)
                count += 1
                if count % 1000 == 0:
                    print("Inserting record {0} into SDE table".format(count))

    # disconnect all users
    print("\nDisconnecting all users...")
    arcpy.DisconnectUser(sde, "ALL")
    # confirm feature class was created
    print("\nUpdated " + outfc)
    
# function to move spatial join data to parcel master staging
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
    # Use list comprehension to build a dictionary from arcpy SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
   
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
#     log.info("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))